# Customer Retention Analysis: Data Exploration & Foundation

**Author:** Harpinder Singh  
**Dataset:** UCI Online Retail II (UK E-Commerce)  
**Objective:** Build foundation for probabilistic CLV modeling and churn prediction

---

## Table of Contents
1. [Environment Setup](#1.-Environment-Setup)
2. [Data Loading & Initial Inspection](#2.-Data-Loading-&-Initial-Inspection)
3. [Data Quality Assessment](#3.-Data-Quality-Assessment)
4. [Data Cleaning & Preprocessing](#4.-Data-Cleaning-&-Preprocessing)
5. [Exploratory Data Analysis](#5.-Exploratory-Data-Analysis)
6. [Temporal Split Creation](#6.-Temporal-Split-Creation)
7. [RFM Feature Engineering](#7.-RFM-Feature-Engineering)
8. [Export Processed Data](#8.-Export-Processed-Data)

---

## 1. Environment Setup

╔════════════════════════════════════════════════════════════════╗
║                    ENVIRONMENT CONFIGURATION                    ║
╚════════════════════════════════════════════════════════════════╝

In [1]:
# Standard library imports
import warnings
from pathlib import Path
from datetime import datetime, timedelta

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Environment configured successfully")
print(f"   Pandas version: {pd.__version__}")
print(f"   NumPy version: {np.__version__}")

✅ Environment configured successfully
   Pandas version: 2.3.3
   NumPy version: 1.26.4


In [2]:
# ┌────────────────────────────────────────────────────────────┐
# │ Directory Structure Setup                                  │
# └────────────────────────────────────────────────────────────┘

# Define project root (handles execution from different directories)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent

# Define all project directories
DIR_DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DIR_DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
DIR_MODELS = PROJECT_ROOT / 'models'
DIR_RESULTS = PROJECT_ROOT / 'results'
DIR_FIGURES = PROJECT_ROOT / 'results' / 'figures'

# Create directories if they don't exist
for directory in [DIR_DATA_PROCESSED, DIR_MODELS, DIR_RESULTS, DIR_FIGURES]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Directory structure verified")
print(f"   Project root: {PROJECT_ROOT}")
print(f"   Raw data: {DIR_DATA_RAW}")
print(f"   Processed data: {DIR_DATA_PROCESSED}")
print(f"   Figures: {DIR_FIGURES}")

✅ Directory structure verified
   Project root: d:\Courses\Workstation\Github\Working Projects\GitHub Project 3 - Customer Churn\Customer_Segmentation_Retention_Analysis
   Raw data: d:\Courses\Workstation\Github\Working Projects\GitHub Project 3 - Customer Churn\Customer_Segmentation_Retention_Analysis\data\raw
   Processed data: d:\Courses\Workstation\Github\Working Projects\GitHub Project 3 - Customer Churn\Customer_Segmentation_Retention_Analysis\data\processed
   Figures: d:\Courses\Workstation\Github\Working Projects\GitHub Project 3 - Customer Churn\Customer_Segmentation_Retention_Analysis\results\figures


---

## 2. Data Loading & Initial Inspection

╔════════════════════════════════════════════════════════════════╗
║                      DATA LOADING                               ║
╚════════════════════════════════════════════════════════════════╝

In [3]:
# ┌────────────────────────────────────────────────────────────┐
# │ Load Raw Data                                              │
# └────────────────────────────────────────────────────────────┘

# File path
file_path = DIR_DATA_RAW / 'Online Retail.xlsx'

print(f"Loading data from: {file_path}")
print("⏳ This may take 30-60 seconds...\n")

# Load Excel file
df_raw = pd.read_excel(file_path, engine='openpyxl')

print(f"✅ Data loaded successfully")
print(f"   Shape: {df_raw.shape}")
print(f"   Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Loading data from: d:\Courses\Workstation\Github\Working Projects\GitHub Project 3 - Customer Churn\Customer_Segmentation_Retention_Analysis\data\raw\Online Retail.xlsx
⏳ This may take 30-60 seconds...

✅ Data loaded successfully
   Shape: (541909, 8)
   Memory usage: 134.93 MB


In [4]:
# ┌────────────────────────────────────────────────────────────┐
# │ Initial Data Inspection                                    │
# └────────────────────────────────────────────────────────────┘

print("Dataset Overview:")
print("="*80)
df_raw.info()

print("\n" + "="*80)
print("First 5 rows:")
print("="*80)
display(df_raw.head())

print("\n" + "="*80)
print("Statistical Summary:")
print("="*80)
display(df_raw.describe())

Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB

First 5 rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom



Statistical Summary:


,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.00,541909,541909.00,406829.00
mean,9.55,2011-07-04 13:34:57.156386048,4.61,15287.69
min,-80995.00,2010-12-01 08:26:00,-11062.06,12346.00
25%,1.00,2011-03-28 11:34:00,1.25,13953.00
50%,3.00,2011-07-19 17:17:00,2.08,15152.00
75%,10.00,2011-10-19 11:27:00,4.13,16791.00
max,80995.00,2011-12-09 12:50:00,38970.00,18287.00
std,218.08,NaN,96.76,1713.60


In [5]:
# ┌────────────────────────────────────────────────────────────┐
# │ Column Analysis                                            │
# └────────────────────────────────────────────────────────────┘

print("Column Details:")
print("="*80)

for col in df_raw.columns:
    print(f"\n{col}:")
    print(f"  Type: {df_raw[col].dtype}")
    print(f"  Non-null: {df_raw[col].notna().sum():,} ({df_raw[col].notna().sum()/len(df_raw)*100:.2f}%)")
    print(f"  Unique: {df_raw[col].nunique():,}")
    
    if df_raw[col].dtype == 'object':
        print(f"  Sample values: {df_raw[col].dropna().unique()[:3].tolist()}")
    else:
        print(f"  Range: [{df_raw[col].min()}, {df_raw[col].max()}]")

Column Details:

InvoiceNo:
  Type: object
  Non-null: 541,909 (100.00%)
  Unique: 25,900
  Sample values: [536365, 536366, 536367]

StockCode:
  Type: object
  Non-null: 541,909 (100.00%)
  Unique: 4,070
  Sample values: ['85123A', 71053, '84406B']

Description:
  Type: object
  Non-null: 540,455 (99.73%)
  Unique: 4,223
  Sample values: ['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER']

Quantity:
  Type: int64
  Non-null: 541,909 (100.00%)
  Unique: 722
  Range: [-80995, 80995]

InvoiceDate:
  Type: datetime64[ns]
  Non-null: 541,909 (100.00%)
  Unique: 23,260
  Range: [2010-12-01 08:26:00, 2011-12-09 12:50:00]

UnitPrice:
  Type: float64
  Non-null: 541,909 (100.00%)
  Unique: 1,630
  Range: [-11062.06, 38970.0]

CustomerID:
  Type: float64
  Non-null: 406,829 (75.07%)
  Unique: 4,372
  Range: [12346.0, 18287.0]

Country:
  Type: object
  Non-null: 541,909 (100.00%)
  Unique: 38
  Sample values: ['United Kingdom', 'France', 'Australia']
